In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

# Combining DataFrames

Di pandas ada 4 cara untuk menggabungkan data dari dataframe yang berbeda:

* Merging
* Joining
* Concatenating 
* Appending

Oke mari kita mulai.

In [2]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


## Merge

Merge adalah untuk melakukan kombinasi kolom yang kompleks dari dataframe dengan cara seperti SQL. Jika Anda tidak tahu gabungan SQL, periksa sumber ini [sql join](https://www.w3schools.com/sql/sql_join.asp).

Dua penggabungan kita membutuhkan dua dataframe, mari kita buat di bawah ini:

In [3]:
tips_bill = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()
tips_tip = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()

del tips_bill['tip']
del tips_tip['total_bill']

In [4]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [5]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

Sekarang kita memiliki dua dataset yang ingin kita gabungkan (alias ambil tipnya dan gabungkan dengan total bill), bagaimana caranya?

In [6]:
pd.merge?

Perhatikan bahwa ada banyak pilihan:

In [7]:
# kita dapat melakukan merge pada indeks
pd.merge(tips_bill, tips_tip, 
         right_index=True, left_index=True)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [8]:
# kita dapat mengatur ulang indeks dan kemudian menggabungkan kolom - mungkin cara termudah
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index(),
    on=['sex', 'smoker']
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [9]:
# ini sebenarnya dapat sama seperti hal di atas - tetapi berhati-hatilah dengan ini
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index()
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [10]:
# dapat menggabungkan sebagian kolom dan indeks
pd.merge(
    tips_bill.reset_index(), 
    tips_tip,
    left_on=['sex', 'smoker'],
    right_index=True
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [11]:
# dapat melakukan kombinasi yang menarik
tips_bill_strange = tips_bill.reset_index(level=0)
tips_bill_strange

,sex,total_bill
smoker,,
Yes,Male,1337.07
No,Male,1919.75
Yes,Female,593.27
No,Female,977.68


In [12]:
pd.merge(
    tips_tip.reset_index(), 
    tips_bill_strange,
    on=['sex', 'smoker']
)

,sex,smoker,tip,total_bill
0,Male,Yes,183.07,1337.07
1,Male,No,302.00,1919.75
2,Female,Yes,96.74,593.27
3,Female,No,149.77,977.68


In [13]:
# kita dapat melakukan fungsionalitas seperti SQL
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='left'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,NaN
3,Female,No,977.68,NaN


In [14]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2),
    how='inner'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00


In [15]:
# dan jika Anda menambahkan indikator...
pd.merge(
    tips_bill.reset_index().tail(3), 
    tips_tip.reset_index().head(3),
    how='outer',
    indicator=True
)

,sex,smoker,total_bill,tip,_merge
0,Male,No,1919.75,302.00,both
1,Female,Yes,593.27,96.74,both
2,Female,No,977.68,NaN,left_only
3,Male,Yes,NaN,183.07,right_only


In [16]:
# dapat menangani kolom dengan nama yang sama
pd.merge(tips_bill, 
         tips_bill, 
         right_index=True, 
         left_index=True,
         suffixes=('_left', '_right')
)

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes              593.27            593.27
       No               977.68            977.68

Ini adalah salah satu bagian pandas yang paling rumit - tetapi sangat penting untuk dikuasai. Jadi silakan periksa latihan di bawah ini!

Satu hal yang harus diperhatikan di sini adalah menggabungkan dua tipe data. String tidak sama dengan int!

# Contatenation

Concatenation adalah untuk menggabungkan lebih dari dua dataframe baik dalam kolom atau baris. Masalah dengan concatenate adalah kombinasi yang memungkinkan Anda melakukannya agak sederhana. Itu sebabnya kita perlu menggabungkan.

Penggabungan dapat mengambil dataframe sebanyak yang Anda inginkan, tetapi memerlukan dataframe yang dibuat secara khusus. Semua dataframe yang Anda berikan harus memiliki indeks yang sama. Jadi tidak perlu lagi menggunakan kolom sebagai indeks.

Mari kita periksa penggunaan dasar di bawah ini:

In [17]:
# ini menambahkan baris dataframe bersama-sama
pd.concat([tips_bill, tips_bill, tips_tip], sort=False)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [18]:
# ini kolomnya
pd.concat([tips_bill, tips_tip], axis=1)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [19]:
# dan akhirnya ini akan menambah dataset dari mana asalnya
pd.concat([tips_bill, tips_tip], sort=False, keys=['num0', 'num1'])

total_bill     tip
     sex    smoker                    
num0 Male   Yes        1337.07     NaN
            No         1919.75     NaN
     Female Yes         593.27     NaN
            No          977.68     NaN
num1 Male   Yes            NaN  183.07
            No             NaN  302.00
     Female Yes            NaN   96.74
            No             NaN  149.77

Seperti yang Anda lihat, tidak ada banyak fungsi untuk digabungkan, tetapi ini sangat berharga jika Anda memiliki lebih dari satu dataframe atau Anda ingin menambahkan baris dari satu dataframe ke yang lain.

## Conclusion

Ada beberapa cara lain untuk menggabungkan data, tetapi mereka cukup terbatas (dan terutama untuk data runtun waktu). 
Diantaranya:

* combine_first
* merge_ordered
* merge_asof